In [5]:
!pip install fredapi


In [15]:
from fredapi import Fred
fred = Fred(api_key="685f7e3646a0a8bdc0deaaea2db1c1f5")
# Monthly real GDP, Treasury yields, unemployment, recession flag:
gdp    = fred.get_series('GDPC1')        # real GDP (monthly interpolated)
spread = fred.get_series('T10Y2Y')       # yield spread
unemp  = fred.get_series('UNRATE')       # unemployment rate
rec    = fred.get_series('USREC')        # 1 if in recession, 0 otherwise


In [16]:
import pandas as pd

# Merge into one DataFrame
df = pd.concat([gdp, spread, unemp, rec], axis=1)
df.columns = ['real_gdp', 'yield_spread', 'unemployment', 'recession']
df.dropna(inplace=True)
df


,real_gdp,yield_spread,unemployment,recession
1976-07-01,6404.895,0.86,7.8,0.0
1976-10-01,6451.177,1.25,7.7,0.0
1977-04-01,6654.466,1.41,7.2,0.0
1977-07-01,6774.457,1.22,6.9,0.0
1979-10-01,7318.535,-0.63,6.0,0.0
...,...,...,...,...
2022-04-01,21919.222,-0.05,3.7,0.0
2022-07-01,22066.784,0.04,3.5,0.0
2024-04-01,23223.906,-0.39,3.9,0.0
2024-07-01,23400.294,-0.29,4.2,0.0


In [17]:
for col in ['yield_spread','unemployment']:
    for lag in [1,3,6]:
        df[f'{col}_lag{lag}'] = df[col].shift(lag)
df.dropna(inplace=True)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(df.drop(columns='recession'))
y = df['recession']
df


,real_gdp,yield_spread,unemployment,recession,yield_spread_lag1,yield_spread_lag3,yield_spread_lag6,unemployment_lag1,unemployment_lag3,unemployment_lag6
1980-07-01,7181.743,1.12,7.8,1.0,-1.90,1.22,0.86,6.9,6.9,7.8
1980-10-01,7315.677,-0.24,7.5,0.0,1.12,-0.63,1.25,7.8,6.0,7.7
1981-04-01,7403.745,-0.05,7.2,0.0,-0.24,-1.90,1.41,7.5,6.9,7.2
1981-07-01,7492.405,-0.90,7.2,0.0,-0.05,1.12,1.22,7.2,7.8,6.9
1981-10-01,7410.768,-0.82,7.9,1.0,-0.90,-0.24,-0.63,7.2,7.5,6.0
...,...,...,...,...,...,...,...,...,...,...
2022-04-01,21919.222,-0.05,3.7,0.0,1.21,1.52,0.39,4.5,6.1,14.8
2022-07-01,22066.784,0.04,3.5,0.0,-0.05,1.23,0.52,3.7,5.4,10.2
2024-04-01,23223.906,-0.39,3.9,0.0,0.04,1.21,0.54,3.5,4.5,6.9
2024-07-01,23400.294,-0.29,4.2,0.0,-0.39,-0.05,1.52,3.9,3.7,6.1


In [19]:
# Create forecast target (next period recession)
df['recession_next'] = df['recession'].shift(-1)
df_forecast = df.dropna()  # drop the last row where recession_next is NaN

# Build new features set (drop both recession columns)
X_new = df_forecast.drop(columns=['recession', 'recession_next'])
y_new = df_forecast['recession_next']

# Standardize features
from sklearn.preprocessing import StandardScaler
scaler_new = StandardScaler()
X_new_scaled = scaler_new.fit_transform(X_new)

# Train logistic regression model for forecast
from sklearn.linear_model import LogisticRegression
model_new = LogisticRegression(max_iter=50000)
model_new.fit(X_new_scaled, y_new)

# Forecast recession probability for the next period using the most recent data
X_recent = X_new.tail(1)
X_recent_scaled = scaler_new.transform(X_recent)
forecast_prob = model_new.predict_proba(X_recent_scaled)[0][1]
latest_date = X_new.tail(1).index[0]
print("Forecast for period starting:", latest_date.date(), 
      "→ Recession probability:", round(forecast_prob, 4))
# Compute and print accuracy on the entire forecast set
from sklearn.metrics import accuracy_score
y_pred = model_new.predict(X_new_scaled)
accuracy = accuracy_score(y_new, y_pred)
print("Forecast Model Accuracy:", accuracy)

Forecast for period starting: 2024-07-01 → Recession probability: 0.2861
Forecast Model Accuracy: 0.8817204301075269
